In [31]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import asyncio

In [27]:
SEASONS = list(range(2019, 2026))

In [28]:
SEASONS

[2019, 2020, 2021, 2022, 2023, 2024, 2025]

In [13]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR)

In [51]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    
    async with async_playwright() as p:
        browser = await p.firefox.launch(headless=False)
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        )
        page = await context.new_page()
        
        try:
            for i in range(1, retries + 1):
                if i > 1:
                    await asyncio.sleep(sleep * i)
                
                try:
                    await page.goto(url, timeout=60000)
                    await page.wait_for_selector(selector, timeout=30000)
                    print(await page.title())
                    html = await page.inner_html(selector)
                    break
                    
                except PlaywrightTimeout:
                    print(f"Timeout error on {url} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                except Exception as e:
                    print(f"Error on {url}: {e} (attempt {i}/{retries})")
                    if i == retries:
                        print(f"Failed after {retries} attempts")
                        
        finally:
            await browser.close()
    
    return html

In [52]:
async def scrape_season(season):
  url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
  html = await get_html(url, "#content .filter")

  soup = BeautifulSoup(html, 'html.parser')
  links = soup.find_all('a')
  href = [l["href"] for l in links]
  standings_pages = [f"https://basketball-reference.com{l}" for l in href]

  for url in standings_pages:
    save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(url, "#all_schedule")
    with open(save_path, "w+") as f:
      f.write(html)

In [53]:
for season in SEASONS:
  await scrape_season(season)

2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com
2022-23 NBA Schedule | Basketball-Reference.com
2023-24 NBA Schedule | Basketball-Reference.com
2024-25 NBA Schedule | Basketball-Reference.com


In [54]:
standings_files =  os.listdir(STANDINGS_DIR)

In [60]:
async def scrape_game(standings_file):
  with open(standings_file, 'r') as f:
    html = f.read()
  
  soup = BeautifulSoup(html)
  links = soup.find_all("a")
  hrefs = [l.get("href") for l in links]
  box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
  box_scores =  [f"https://www.basketball-reference.com{l}" for l in box_scores]

  for url in box_scores:
    save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
    if os.path.exists(save_path):
      continue

    html = await get_html(url, "#content")
    if not html:
      continue
    with open(save_path, "w+") as f:
      f.write(html)

In [ ]:
for f in standings_files:
  filepath = os.path.join(STANDINGS_DIR, f)

  await scrape_game(filepath) 